In [1]:
suits = ("Hearts","Clubs","Diamonds","Spades")
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 
            'Nine':9, 'Ten':10, 'Jack':10, 'Queen':10, 'King':10, 'Ace':"A"}
from random import shuffle

In [2]:
# Creating a player class where the player
# Is going to be an instance from this class

class Player():
    
    def __init__(self,name,balance,player_cards = []):
        self.name = name
        self.balance = balance
        self.cards = player_cards
    
    
    # Print player's name and total balance
    
    def __str__(self):
        print(f"{self.name}.capitalize() has {self.balance}$ to bet.")
    
    
    # Betting Method
    
    def player_bet(self):
        while True:
            try:
                bet = int(input("Place your bet sir: "))
            except:
                print("Please enter your betting amount using numbers from your keyboard")
            else:
                if bet > self.balance:
                    print("Sorry, you dont have sufficient funds to make this bet. Please\n try betting a smaller amount.")
                else:
                    print("Your bet of {}$ was succesffuly placed!".format(bet))
                    self.balance -= bet
                    break
        return bet
            

In [3]:
# Creating cards

class Card():
    
    def __init__(self,suit,rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
        
    
    def __str__(self):
        return (f"{self.rank} of {self.suit}")

In [4]:
# Creating a deck of cards

class DeckOfCards():
    
    def __init__(self):
        self.all_cards = []
        for suit in suits:
            for rank in ranks:
                self.all_cards.append(Card(suit,rank))
    
    def shuffle(self):
        shuffle(self.all_cards)
    
    # Initial deal to the player and to the dealer
    def first_deal(self):
        player_first_card = self.all_cards.pop()
        player_second_card = self.all_cards.pop()
        dealers_first_card = self.all_cards.pop()
        dealers_second_card = self.all_cards.pop()
        return player_first_card,player_second_card,dealers_first_card,dealers_second_card
    
    
    # Normal deal to both the player and the dealer
    
    def deal(self):
        return self.all_cards.pop()
        

In [5]:
def table_display(players_cards = [],dealers_cards = []):
    player_sum = 0
    dealer_sum = 0
    print("----------------------")
    print("   Dealer   ")
    for item in dealers_cards:
        print(f"   {item}   ")
        if item.value == "A" and dealer_sum + 11 <= 21:
            dealer_sum += 11
        elif item.value == "A" and dealer_sum + 11 > 21:
            dealer_sum += 1
        else:
            dealer_sum += item.value
    if dealer_sum == 21:
        print("   Blackjack  ")
    print("----------------------")
    for item in players_cards:
        print(f"   {item}   ")
        if item.value == "A" and player_sum + 11 <= 21:
            player_sum += 11
        elif item.value == "A" and player_sum + 11 > 21:
            player_sum += 1
        else:
            player_sum += item.value
    if player_sum == 21:
        print("   Blackjack!  ")
    print("----------------------")
    print(f"You are at {player_sum} and the dealer is at {dealer_sum}")
    return (player_sum,dealer_sum)

In [6]:
# Blackjack Main script


# Initial values
game_on = True
dealers_cards = []


print("Welcome sir to the Blackjack table. Please sit down and let the game begin!")
# Ask for players name and balance
print("Please give us your name and the amount of money you will be betting with")
name = input("Your name is...:").title()
while True:
    try:
        balance = int(input(f"Mr {name} provide the available funds that you will be betting: "))
    except:
        print("Sorry, some error occured while providing your available balance. Please try again")
    else:
        break
human_player = Player(name,balance)
deck = DeckOfCards()

# Start of the rounds
first_turn = True
while game_on:
    print(f"The round begins!! Your balance is {human_player.balance}.")
    if first_turn:
        bet = human_player.player_bet()
        deck.shuffle()
        
        # First deal after betting
        player_first_card,player_second_card,dealers_first_card,dealers_second_card = deck.first_deal()
        human_player.cards.append(player_first_card)
        human_player.cards.append(player_second_card)
        dealers_cards.append(dealers_first_card)
        first_turn = False
        player_sum,dealer_sum = table_display(human_player.cards,dealers_cards)
    else:
        while True:
            player_answer = input("Please decide if you are going to Hit or Stand\n by typing H for Hit and S for Stand: ").upper()
            if player_answer not in ["H",'S']:
                print("Sorry, wrong input. Please give an H for Hit or S for Stand.")
            else:
                break
        if player_answer == 'H':
            human_player.cards.append(deck.deal())
            player_sum,dealer_sum = table_display(human_player.cards,dealers_cards)
            if player_sum > 21 :
                print("BUST!! House wins!")
                human_player.cards = []
                dealers_cards = []
        else:
            dealers_cards.append(dealers_second_card)
            player_sum,dealer_sum = table_display(human_player.cards,dealers_cards)
            while True:
                if dealer_sum > player_sum and dealer_sum <= 21:
                    print("The house wins!")
                    human_player.cards = []
                    dealers_cards = []
                    break
                elif dealer_sum > 21:
                    print("The house has busted!!! You win!")
                    human_player.balance += 2*bet
                    human_player.cards = []
                    dealers_cards = []
                    break
                elif dealer_sum == player_sum and player_sum == 21:
                    print("DRAW!!!")
                    human_player.balance += bet
                    human_player.cards = []
                    dealers_cards = []
                    break
                dealers_cards.append(deck.deal())
                player_sum,dealer_sum = table_display(human_player.cards,dealers_cards)
    if human_player.cards == []:
        while True:
            keep_playing = input("To keep playing type Y and to stop type N. If you want to keep playing\n and add funds type A: ").upper()
            if keep_playing not in ['Y','N','A']:
                print("Wrong input.")
            elif keep_playing == "N":
                game_on = False
                break
            elif keep_playing == 'A':
                while True:
                    try:
                        add_funds = int(input("Type the amount you want added: "))
                    except:
                        print('Wrong input')
                    else:
                        human_player.balance += add_funds
                        break
            elif keep_playing == 'Y' and human_player.balance > 0:
                first_turn = True
                break
            else:
                print('You have no more funds to bet to keep playing add funds.')
                while True:
                    try:
                        add_funds = int(input("Type the amount you want added: "))
                    except:
                        print('Wrong input')
                    else:
                        human_player.balance += add_funds
                        first_turn = True
                        break
                break

Welcome sir to the Blackjack table. Please sit down and let the game begin!
Please give us your name and the amount of money you will be betting with
Your name is...:Vangs
Mr Vangs provide the available funds that you will be betting: 250
The round begins!! Your balance is 250.
Place your bet sir: 50
Your bet of 50$ was succesffuly placed!
----------------------
   Dealer   
   Eight of Clubs   
----------------------
   Three of Clubs   
   Ten of Diamonds   
----------------------
You are at 13 and the dealer is at 8
The round begins!! Your balance is 200.
Please decide if you are going to Hit or Stand
 by typing H for Hit and S for Stand: h
----------------------
   Dealer   
   Eight of Clubs   
----------------------
   Three of Clubs   
   Ten of Diamonds   
   King of Clubs   
----------------------
You are at 23 and the dealer is at 8
BUST!! House wins!
To keep playing type Y and to stop type N. If you want to keep playing
 and add funds type A: y
The round begins!! Your balanc